# Python Process

## Preparing necessary modules

In [21]:
!nvidia-smi
!pip install tensorflowjs

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten, GRU ## Neural networks layers 
from tensorflow.keras.models import Sequential
from tensorflow.keras import utils
from tensorflow.keras.optimizers import SGD, RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import tensorflowjs as tfjs
import tensorflow as tf
import pandas as pd
import numpy as np
import re, json

## Exploring dataset used

In [2]:
dataset = pd.read_csv('https://raw.githubusercontent.com/Boubker-1/hate-speech-detection/main/misogyny.csv', sep ='\t')
data = dataset.drop(['tweet_id', 'misogyny'], axis=1)

In [3]:
dataset

,tweet_id,text,misogyny,category
0,0,مستخدم@ وجعجع رئيسك رجل شو؟ ؟؟,none,none
1,1,مستخدم@ انتي مصدقه انك لكي اي لازمه انتي بنسبه...,misogyny,discredit
2,2,مستخدم@ متل كعب صباطك شعب رتش,none,none
3,3,مستخدم@ مستخدم@ مستخدم@ ليش الهجوم عليها؟ هي ع...,none,none
4,4,مستخدم@ اهلا وسهلا فيكي بطرابلس,none,none
...,...,...,...,...
7861,7861,RT مستخدم@: مستخدم@ اسمعي يا فطيرة الكشك ما حد...,misogyny,discredit
7862,7862,مستخدم@ مستخدم@ معلومات عامة: لا يوجد ممثل للح...,none,none
7863,7863,مستخدم@ أسأل الله ان يجعل يدك مشلوله وارجلك مع...,misogyny,damning
7864,7864,مستخدم@ الثورة عطتك عمرها يا معترة هههه,misogyny,discredit


In [4]:
data['category'].value_counts()

none                              3061
discredit                         2868
damning                            669
stereotyping & objectification     653
threat of violence                 230
dominance                          219
derailing                          105
sexual harassment                   61
Name: category, dtype: int64

In [5]:
df = data.rename(columns = {'text' : 'Text', 'category' : 'Label'}, inplace = False)
df.Label.replace({'none':0, 'discredit':1, 'damning':2, 'stereotyping & objectification':3, 'threat of violence':4, 'dominance': 5, 'derailing': 6, 'sexual harassment':7}, inplace = True)
df.count()

Text     7866
Label    7866
dtype: int64

## Preprocessing our dataset

In [6]:
def clean_text(text):
    no_mentions = re.sub('([\u0600-\u06FF]+)@', '', text)
    no_underscore = re.sub(r'_', ' ', no_mentions)
    lines = no_underscore.split("\n")
    filtered_lines = [re.sub(r'[^\u0600-\u06FF ]', '', line) for line in lines if line is not ""]
    filtered =  '\n'.join(filtered_lines);
    no_diacritics = re.sub(r'[^\u0621-\u063A\u0641-\u064A\u0660-\u0669a-zA-Z 0-9]', '', filtered)
    no_punctuations = re.sub(r'،؛؟«»!', '', no_diacritics)
    clean = re.sub(r'(.)\1+', r'\1', no_punctuations) # remove repeated characters
    return clean.lstrip().rstrip()

<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\MSI\AppData\Local\Temp\ipykernel_7220\3045270088.py:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  filtered_lines = [re.sub(r'[^\u0600-\u06FF ]', '', line) for line in lines if line is not ""]


In [7]:
df['Text'][4100:4120].values

array(['مستخدم@ لله درك يا بنت الحالمة وحمامة السلام السيدة توكل كرمان الف تحية وإجلال وتقدير لك أختي',
       'مستخدم@ لعنت الله عليك توكل كرمان دمرتي بلادي من أجل مصالحك ومصالح الغرب يا كذابين',
       'مستخدم@ مستخدم@ نشالله يارب وما تلاقي مين يلمك يا ديما الواطيه',
       'مستخدم@ وجهي كلامك لحسن نصرالله ولنبيه بري مش هني المسؤلين عن العصابة اللي نزلوا!!!! للأسف عصابة بري ونصرالله أقوى من الدولة.',
       'مستخدم@ يكهربوكي ببريييييز',
       'مستخدم@ والله ما في اضل منك يا بهيمة ويا بنت البهائم اتفوه علي وجهك المقزز يا مشعودة',
       'مستخدم@ الله يعين المجتمع ع قلة فهمك و استيعابك �',
       'مستخدم@ لعمى بش بس وقحه كذابه كمان',
       'مستخدم@ مستخدم@ انت الواطيه تفو عليك',
       'مستخدم@ ولعيونك #كلنا_ديما_صادق �� https://t.co/aTWdgkPQCp',
       'مستخدم@ " حمالة الحطب اول من كسرها بتأمرها مع اذناب الفرس بالانقلاب على الشرعية',
       'مستخدم@ يا هبلة لو مناصرين الحزب ترك من تحت ما بضل حدا',
       'مستخدم@ وخري ريحتك ماعز 🤣 الي مثلك لازم يعرف انه بالنسبه للغرب مجرد عرائس يقوم

In [8]:
df.Text = df.Text.apply(clean_text)
df['Text'][4100:4120].values

array(['له درك يا بنت الحالمة وحمامة السلام السيدة توكل كرمان الف تحية وإجلال وتقدير لك أختي',
       'لعنت اله عليك توكل كرمان دمرتي بلادي من أجل مصالحك ومصالح الغرب يا كذابين',
       'نشاله يارب وما تلاقي مين يلمك يا ديما الواطيه',
       'وجهي كلامك لحسن نصراله ولنبيه بري مش هني المسؤلين عن العصابة الي نزلوا لأسف عصابة بري ونصراله أقوى من الدولة',
       'يكهربوكي بريز',
       'واله ما في اضل منك يا بهيمة ويا بنت البهائم اتفوه علي وجهك المقز يا مشعودة',
       'اله يعين المجتمع ع قلة فهمك و استيعابك',
       'لعمى بش بس وقحه كذابه كمان', 'انت الواطيه تفو عليك',
       'ولعيونك كلنا ديما صادق',
       'حمالة الحطب اول من كسرها بتأمرها مع اذناب الفرس بالانقلاب على الشرعية',
       'يا هبلة لو مناصرين الحزب ترك من تحت ما بضل حدا',
       'وخري ريحتك ماعز الي مثلك لازم يعرف انه بالنسبه لغرب مجرد عرائس يقومو بتحريكم مثلما اشائو وانتي وامثالك ستظل دماء الشعوب تطاردكم وارواح الاطفال لابد',
       'أنت الأقوى كلنا ديما صادق',
       'ادعوا كل الثوار لتوجه الى ساحة الشهداء',
       'وإنت أ

In [9]:
X = df['Text'].astype(str).values
labels = df['Label'].values

y = utils.to_categorical(labels)

X_tr_temp, X_val, y_train_temp, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
tuples, classes = [], []
for i in range(len(y)):
  if labels[i] not in classes:
    classes.append(labels[i])
    tuples.append((labels[i], [int(j) for j in y[i]]))

tuples.sort()
tuples

[(0, [1, 0, 0, 0, 0, 0, 0, 0]),
 (1, [0, 1, 0, 0, 0, 0, 0, 0]),
 (2, [0, 0, 1, 0, 0, 0, 0, 0]),
 (3, [0, 0, 0, 1, 0, 0, 0, 0]),
 (4, [0, 0, 0, 0, 1, 0, 0, 0]),
 (5, [0, 0, 0, 0, 0, 1, 0, 0]),
 (6, [0, 0, 0, 0, 0, 0, 1, 0]),
 (7, [0, 0, 0, 0, 0, 0, 0, 1])]

In [11]:
max_length = 250
EMBEDDING_DIM = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_tr_temp)
vocab_size = len(tokenizer.word_index) + 1

tokenized_train = tokenizer.texts_to_sequences(X_tr_temp)
tokenized_test = tokenizer.texts_to_sequences(X_val)

X_tr = pad_sequences(tokenized_train, maxlen=max_length)
X_test = pad_sequences(tokenized_test, maxlen=max_length)



smote = SMOTE(sampling_strategy="minority")
X_train, y_train = smote.fit_resample(X_tr, y_train_temp)

## Machine Learning process

In [15]:
tf.keras.backend.clear_session()

In [12]:
model=Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length)) 

model.add(Bidirectional(LSTM(units=max_length,return_sequences=True)))

model.add(Flatten())

model.add(Dense(400, activation='relu'))

# The final ( output ) Layer

model.add(Dense(8, activation='softmax'))

# softmax : to handle the output ( categorical case )

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
# categorical_crossentropy : because we have a categorical classes classification task 
# Adam : Stochastic gradient decenet optimizatiion 
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          2058400   
                                                                 
 bidirectional (Bidirectiona  (None, 250, 500)         702000    
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 125000)            0         
                                                                 
 dense (Dense)               (None, 400)               50000400  
                                                                 
 dense_1 (Dense)             (None, 8)                 3208      
                                                                 
Total params: 52,764,008
Trainable params: 52,764,008
Non-trainable params: 0
____________________________________________

In [13]:
model.fit(X_train, y_train, validation_data=(X_test, y_val), epochs=5, batch_size=128)

Epoch 1/5
68/68 [==============================] - 13s 147ms/step - loss: 1.5602 - accuracy: 0.4120 - val_loss: 1.1892 - val_accuracy: 0.6258
Epoch 2/5
68/68 [==============================] - 9s 139ms/step - loss: 0.7242 - accuracy: 0.7709 - val_loss: 1.1117 - val_accuracy: 0.6671
Epoch 3/5
68/68 [==============================] - 9s 139ms/step - loss: 0.1929 - accuracy: 0.9401 - val_loss: 1.3717 - val_accuracy: 0.6493
Epoch 4/5
68/68 [==============================] - 9s 139ms/step - loss: 0.0722 - accuracy: 0.9819 - val_loss: 1.8160 - val_accuracy: 0.6525
Epoch 5/5
68/68 [==============================] - 10s 140ms/step - loss: 0.0281 - accuracy: 0.9923 - val_loss: 2.0366 - val_accuracy: 0.6601


In [19]:
count = 0
value = len(X_test)
correct = []
for sent_index in range(value):
  instance = pad_sequences(tokenizer.texts_to_sequences([X_val[sent_index]]), maxlen=max_length)
  prediction = model.predict(instance)[0]
  real = y_val[sent_index]
  pred_index = np.argmax(prediction)
#  print((prediction[pred_index].round(), real[pred_index]))
  if (prediction[pred_index].round() == real[pred_index]):
    correct.append([[X_val[sent_index]], pred_index, ])
#    print("%dth example: Correct Classification" % (sent_index))
    count += 1
#  else:
#    print("%dth example: Wrong Classification" % (sent_index))

print("Number of correct classes out of all: %d/%d: " % (count, value))
#  comparison = [(prediction[i], real[i]) for i in range(len(prediction))]
#  print(X_val[sent_index])
#  print(comparison)

Number of correct classes out of all: 1062/1574: 


In [18]:
for i in range(len(X_val)):
    print(X_val[i])

تافهة وراح تضلك تافهة
لا تقتربي كثير من هيلاري فهي تستاء من رائحتك التي تشبه رائحه الماعزوان ابتسمت في وجهك
نباح بلا حدود ديما الشرشوحة
بما انك قليلة تهذيب انت خيارك تكوني بهيمة
ماعندها زوج او اهل يعلموها الادب ويلموها في بيت
ولعيونك كلنا ديما صادق
واله ما غيرك نجس الثورة اكشفي عن زوجك يلي عنده شغل مع غازي زعيتر يا فاسدة
لعنة اله عليش لولا الحوثي ماجات السعوديه ولا الامارات
يا حقيرة حضري لفيديو منيح الشاب ما ضربا لبنت ونصيحه ما تلعبي مع بيت شمص انتي مش ادون يا حشرة
بس يحرك اصبعو السيد منصير نشوف اله بل سما وعالارض ما منشوف حدا ومنحرق اختو
حضرتك اوسخ من الوسخ نفسه
يا بنت الحرام انضبي
انتى اشتغلى ايه لابيض ولاصفر انتى جربوعة عاوز عيش بلاش بصى على بلدك ودورى على نفسين حشيشة وبعدين اتكلمى على جيش انتى كلة حقية
يا ثوار المحن قوموا علنسوان
ان شاله بيقبر قلبك هو قاعد بقلبنا ومتربع والامر الو وبس سديه يا
روحي انضبي احسن
يا حبيبتي يا نوال موقعك مش تلفزيون المر انتي مقاومة ومن بيت ابو المقاومة وحاميها والي بيغير ث
عنز اليمن التي رائحتها كريهة جدا شمطاء اليمن رسائل هيلاري كلينتون سوال من شمطاء ال

In [14]:
y_test= model.predict(X_test)
y_test= np.round(y_test, 0).astype(int)

print('\t\t\tPERFORMANCE\n')
print('Accuracy:', round(accuracy_score(y_val, y_test), 4), '\n')
print(classification_report(y_val, y_test))
cmatrix = confusion_matrix(y_val.argmax(axis=1), y_test.argmax(axis=1))
print(cmatrix)

			PERFORMANCE

Accuracy: 0.6499 

              precision    recall  f1-score   support

           0       0.81      0.74      0.78       624
           1       0.70      0.67      0.69       550
           2       0.74      0.72      0.73       132
           3       0.67      0.56      0.61       129
           4       0.29      0.15      0.20        53
           5       0.43      0.17      0.24        53
           6       0.14      0.05      0.08        19
           7       0.02      0.21      0.04        14

   micro avg       0.67      0.65      0.66      1574
   macro avg       0.48      0.41      0.42      1574
weighted avg       0.71      0.65      0.68      1574
 samples avg       0.65      0.65      0.65      1574

[[481  74   5   9   5   2   0  48]
 [ 87 371  14  17   4   4   2  51]
 [  7  16  95   2   5   0   1   6]
 [ 15  27   2  72   1   2   1   9]
 [ 17  15   7   0   8   2   0   4]
 [ 12  16   0   7   3   9   2   4]
 [  6   4   3   0   2   1   1   2]
 [  3   5   2  

C:\Users\MSI\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Pushing model architectures and weights to Git repository

In [ ]:
#model.save("model.h5")
#new_model = tf.keras.models.load_model('model.h5')
#!tensorflowjs_converter --input_format keras model.h5 model
#!zip -r model.zip model

In [15]:
tfjs.converters.save_keras_model(model, 'model')

In [16]:
import json
with open('vocabulary.json', 'w', encoding="utf-8") as json_file:
  json.dump(tokenizer.word_index, json_file, ensure_ascii = False)